In [114]:
import os
import requests
from Bio import PDB

In [124]:
### website requests ###

atpbind = 'https://seq2fun.dcmb.med.umich.edu//ATPbind/'
pdb_ext = []
with open('PHAS0052 PDB File - Sheet1.csv') as csvDataFile:
    read = csv.reader(csvDataFile)
    for i in read:
        pdb_ext.append(i[0])

pdb_info = [(entry[:-1].lower(), entry[-1]) for entry in pdb_ext]

print(pdb_info)

[('6ksh', 'D'), ('4cta', 'A'), ('2x14', 'A'), ('2j9c', 'A'), ('3f5m', 'A'), ('3ruv', 'D'), ('5trd', 'A'), ('3c1m', 'C'), ('3v2u', 'C'), ('1f3f', 'C'), ('7nsd', 'A'), ('5dd7', 'A'), ('1i58', 'A'), ('3hy2', 'Y'), ('7uld', 'A'), ('1z0s', 'A'), ('1un9', 'A'), ('7alr', 'A'), ('121p', 'A'), ('2jg1', 'C'), ('6cau', 'A'), ('2py7', 'X'), ('5dgh', 'A'), ('2aqx', 'B'), ('3dnt', 'A'), ('3wgu', 'C'), ('4uxx', 'C'), ('2xan', 'A'), ('6ci7', 'C'), ('3gqk', 'A'), ('4amf', 'A'), ('1d4x', 'A'), ('7cqq', 'A'), ('4lac', 'C'), ('7d8i', 'A'), ('1xdp', 'A'), ('2i1o', 'A'), ('1vl1', 'A'), ('1to6', 'A'), ('1yzy', 'A'), ('1fit', 'A'), ('1s1d', 'A'), ('6a8p', 'B'), ('2bz0', 'A'), ('3zcb', 'A'), ('4edk', 'A'), ('5guf', 'A'), ('1k90', 'A'), ('4crj', 'A'), ('6aaz', 'A'), ('6fl4', 'A'), ('8dcd', 'A'), ('1twf', 'B'), ('7tgk', 'D'), ('1rn8', 'A'), ('1j09', 'A'), ('2q16', 'B'), ('7edz', 'C'), ('1wc6', 'B'), ('3tux', 'A'), ('3vth', 'A'), ('6p1p', 'A'), ('6r5d', 'A'), ('4yvz', 'A'), ('4ru9', 'A'), ('3f2b', 'A'), ('3erc', 

In [126]:
import requests
import os
from Bio import PDB
import time

# ATPbind URL
URL = "https://zhanggroup.org/ATPbind/atpbind.cgi"
EMAIL = "angelineguoatp@gmail.com"  # Replace with your actual email

# PDB downloader
pdbl = PDB.PDBList()

# Function to extract chain from downloaded PDB file
def extract_chain(pdb_id, chain_id):
    pdbl.retrieve_pdb_file(pdb_id, file_format="pdb", pdir=".")
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure(pdb_id, f"pdb{pdb_id}.ent")

    output_filename = f"{pdb_id}_chain_{chain_id}.pdb"
    with open(output_filename, "w") as out_f:
        for model in structure:
            for chain in model:
                if chain.id == chain_id:
                    io = PDB.PDBIO()
                    io.set_structure(chain)
                    io.save(out_f)
                    return output_filename
    return None

# Function to check if ATPbind allows new submissions
def can_submit_job():
    response = requests.post(URL, data={"REPLY-E-MAIL": EMAIL, "TARGET-NAME": "test_job"})
    if "10 jobs already submitted" in response.text:
        return False  # Job limit reached
    return True  # Submission allowed

# Submit each extracted chain to ATPbind
for pdb_id, chain in pdb_info:
    while not can_submit_job():
        print("⚠️ 10 jobs are already running. Waiting 5 minutes before retrying...")
        time.sleep(300)  # Wait 5 minutes before checking again

    extracted_chain_file = extract_chain(pdb_id, chain)

    if not extracted_chain_file:
        print(f"❌ Failed to extract Chain {chain} from {pdb_id}, skipping...")
        continue

    with open(extracted_chain_file, "rb") as f:
        response = requests.post(
            URL,
            files={"str_file": (extracted_chain_file, f, "chemical/x-pdb")},
            data={"REPLY-E-MAIL": EMAIL, "TARGET-NAME": pdb_id},
        )

    if response.status_code == 200:
        print(f"✅ {extracted_chain_file} submitted successfully!")
    else:
        print(f"❌ Failed to submit {extracted_chain_file} (HTTP {response.status_code})")


Structure exists: './pdb6ksh.ent' 
✅ 6ksh_chain_D.pdb submitted successfully!
Structure exists: './pdb4cta.ent' 
✅ 4cta_chain_A.pdb submitted successfully!
Structure exists: './pdb2x14.ent' 
✅ 2x14_chain_A.pdb submitted successfully!
Structure exists: './pdb2j9c.ent' 
✅ 2j9c_chain_A.pdb submitted successfully!
Structure exists: './pdb3f5m.ent' 
✅ 3f5m_chain_A.pdb submitted successfully!
Structure exists: './pdb3ruv.ent' 
✅ 3ruv_chain_D.pdb submitted successfully!
Structure exists: './pdb5trd.ent' 
✅ 5trd_chain_A.pdb submitted successfully!
Structure exists: './pdb3c1m.ent' 
✅ 3c1m_chain_C.pdb submitted successfully!
Structure exists: './pdb3v2u.ent' 
✅ 3v2u_chain_C.pdb submitted successfully!
Structure exists: './pdb1f3f.ent' 
✅ 1f3f_chain_C.pdb submitted successfully!
Structure exists: './pdb7nsd.ent' 
✅ 7nsd_chain_A.pdb submitted successfully!
Structure exists: './pdb5dd7.ent' 
✅ 5dd7_chain_A.pdb submitted successfully!
Structure exists: './pdb1i58.ent' 
✅ 1i58_chain_A.pdb submitted 

KeyboardInterrupt: 